In [9]:
# !pip install pandas

In [1]:
import numpy as np
import pandas as pd
from pandas import DataFrame as df
import datetime as dt
from datetime import timedelta
import json
import os
import csv
import matplotlib.pyplot as plt


In [2]:
localdir=os.getcwd()
# localdir

In [106]:
folder_list=[folder for folder in os.listdir(localdir) if 'Agents_' in folder]
print(len(folder_list), os.listdir(folder_list[0]))

24 ['2024-09-05-9.log', '2024-09-05-9.log.gz', 'Agents_A10.json']


In [107]:
# for every game, find no. of whisper & broadcast & total chats happening
BROADCAST = 'broadcast'
WHISPER = 'whisper'
players_list = ['Alan', 'Kalyani', 'Lamar', 'Sarah', 'Sokka']

game_chat_dict= {}
player_chat_count_dict = {}
player_chat_lines_dict = {}
print(player_chat_count_dict)

required_dict = {'gametype': [], 'game': [], 'player':[], 'mode':[], 'line':[]}
gametype_list = []
game_list = []
player_list = []
mode_list = []
line_list = []

for folder in folder_list:
    total_broadcast = 0
    total_whisper = 0
    print(folder)
    player_chat_lines_dict[folder] = {player: {BROADCAST: [], WHISPER: []} for player in players_list}
    game_chat_dict[folder]= {WHISPER:[], BROADCAST:[]}
    player_chat_count_dict[folder] = {player: {BROADCAST: 0, WHISPER: 0} for player in players_list}
    chat_file=[item for item in os.listdir(folder) if '.log' in item and '.gz' not in item][0]
    print(chat_file)
    chatfile=open(os.path.join(localdir,folder,chat_file), encoding='utf8')
    lines=chatfile.readlines()
    chatfile.close()

    #whisper_list=[]
    #notsecure_list=[]

    for line in lines:
        if "[private" in line:
            game_chat_dict[folder][WHISPER].append(line)
            if ': §f[private' in line:
                player_name = line.split(': §f[private')[0].split('] ')[-1]
            else:
                player_name = line.split(': [private')[0].split('] ')[-1]
            # print(player_name, line)
            player_chat_count_dict[folder][player_name][WHISPER] += 1
            player_chat_lines_dict[folder][player_name][WHISPER].append(line)
            total_whisper += 1

            game_list.append(folder.split('_')[1])
            if 'A' in folder.split('_')[1]:
                gametype_list.append('Competitive')
            elif 'B' in folder.split('_')[1]:
                gametype_list.append('Baseline')
            else:
                gametype_list.append('Cooperative')
            player_list.append(player_name)
            mode_list.append(WHISPER)
            line_list.append(line)
        
        if "<Kalyani>" in line or "<Sarah>" in line or "<Sokka>" in line or "<Alan>" in line or "<Lamar>" in line:
            game_chat_dict[folder][BROADCAST].append(line)
            player_name = line.split('[BarterPlus] <')[1].split('>')[0]
            line_ = line.split(player_name+'> ')[1]
            # print(player_name)
            player_chat_count_dict[folder][player_name][BROADCAST] += 1
            player_chat_lines_dict[folder][player_name][BROADCAST].append(line)
            total_broadcast += 1

            game_list.append(folder.split('_')[1])
            if 'A' in folder.split('_')[1]:
                gametype_list.append('Competitive')
            elif 'B' in folder.split('_')[1]:
                gametype_list.append('Baseline')
            else:
                gametype_list.append('Cooperative')
            player_list.append(player_name)
            mode_list.append(BROADCAST)
            line_list.append(line)

    for player_name in players_list:
        player_chat_count_dict[folder][player_name][BROADCAST] -= 15
        player_chat_lines_dict[folder][player_name][BROADCAST] = player_chat_lines_dict[folder][player_name][BROADCAST][:-15]
    total_broadcast -= 75
    print('total_broadcast = ', total_broadcast)
    print('total_whisper = ', total_whisper)
    dms= len(game_chat_dict[folder][WHISPER])
    print('no.of whisper chats = ', dms)
    globe= len(game_chat_dict[folder][BROADCAST])-75
    print('no. of broadcasts = ', globe)
    print('total no. of chats = ', dms+globe)
    print('whisper chats (% of total) = ', round(dms/(dms+globe) *100, 2))
    print(player_chat_count_dict[folder])    
    print('##################')

required_dict = {'gametype': gametype_list, 'game': game_list, 'player': player_list,
                 'mode': mode_list, 'line': line_list}
result_df = df.from_dict(required_dict)

{}
Agents_A10
2024-09-05-9.log
total_broadcast =  58
total_whisper =  98
no.of whisper chats =  98
no. of broadcasts =  58
total no. of chats =  156
whisper chats (% of total) =  62.82
{'Alan': {'broadcast': 1, 'whisper': 15}, 'Kalyani': {'broadcast': 17, 'whisper': 19}, 'Lamar': {'broadcast': 23, 'whisper': 20}, 'Sarah': {'broadcast': -2, 'whisper': 17}, 'Sokka': {'broadcast': 19, 'whisper': 27}}
##################
Agents_A3
2024-09-04-9.log
total_broadcast =  126
total_whisper =  84
no.of whisper chats =  84
no. of broadcasts =  126
total no. of chats =  210
whisper chats (% of total) =  40.0
{'Alan': {'broadcast': 31, 'whisper': 14}, 'Kalyani': {'broadcast': 4, 'whisper': 22}, 'Lamar': {'broadcast': 43, 'whisper': 0}, 'Sarah': {'broadcast': 23, 'whisper': 21}, 'Sokka': {'broadcast': 25, 'whisper': 27}}
##################
Agents_A4
2024-09-04-11.log
total_broadcast =  159
total_whisper =  34
no.of whisper chats =  34
no. of broadcasts =  159
total no. of chats =  193
whisper chats (%

In [108]:
result_df[result_df['mode']==WHISPER]

,gametype,game,player,mode,line
1,Competitive,A10,Alan,whisper,[22:33:23] [Craft Scheduler Thread - 15 - Bart...
3,Competitive,A10,Alan,whisper,[22:33:49] [Craft Scheduler Thread - 18 - Bart...
6,Competitive,A10,Sarah,whisper,[22:34:35] [Craft Scheduler Thread - 22 - Bart...
7,Competitive,A10,Sokka,whisper,[22:34:41] [Craft Scheduler Thread - 23 - Bart...
8,Competitive,A10,Lamar,whisper,[22:34:57] [Craft Scheduler Thread - 24 - Bart...
...,...,...,...,...,...
6592,Cooperative,C8,Sokka,whisper,[14:38:00] [Craft Scheduler Thread - 152 - Bar...
6593,Cooperative,C8,Sokka,whisper,[14:38:00] [Craft Scheduler Thread - 152 - Bar...
6595,Cooperative,C8,Sokka,whisper,[14:38:18] [Craft Scheduler Thread - 154 - Bar...
6596,Cooperative,C8,Kalyani,whisper,[14:38:21] [Craft Scheduler Thread - 155 - Bar...


In [109]:
result_df.shape

(6686, 5)

In [110]:
survey_responses_indices_list = []

for folder in folder_list:
    for player in players_list:
        print('###########', folder, player)
        gamename = folder.split('_')[1]
        game_df = result_df[result_df['game']==gamename]
        game_player_df = game_df[game_df['player']==player]
        game_player_broadcast_df = game_player_df[game_player_df['mode']==BROADCAST]   
        survey_responses_indices_list = survey_responses_indices_list + game_player_broadcast_df.index[-15:].to_list()


########### Agents_A10 Alan
########### Agents_A10 Kalyani
########### Agents_A10 Lamar
########### Agents_A10 Sarah
########### Agents_A10 Sokka
########### Agents_A3 Alan
########### Agents_A3 Kalyani
########### Agents_A3 Lamar
########### Agents_A3 Sarah
########### Agents_A3 Sokka
########### Agents_A4 Alan
########### Agents_A4 Kalyani
########### Agents_A4 Lamar
########### Agents_A4 Sarah
########### Agents_A4 Sokka
########### Agents_A5 Alan
########### Agents_A5 Kalyani
########### Agents_A5 Lamar
########### Agents_A5 Sarah
########### Agents_A5 Sokka
########### Agents_A6 Alan
########### Agents_A6 Kalyani
########### Agents_A6 Lamar
########### Agents_A6 Sarah
########### Agents_A6 Sokka
########### Agents_A7 Alan
########### Agents_A7 Kalyani
########### Agents_A7 Lamar
########### Agents_A7 Sarah
########### Agents_A7 Sokka
########### Agents_A8 Alan
########### Agents_A8 Kalyani
########### Agents_A8 Lamar
########### Agents_A8 Sarah
########### Agents_A8 Sokka
########

In [111]:
result_df.drop(index=survey_responses_indices_list, inplace=True)
result_df.shape, len(survey_responses_indices_list), result_df.shape[0] + len(survey_responses_indices_list), 

((4891, 5), 1795, 6686)

In [112]:
ego_words_list = ['I\'m ', 'I need ', 'I want ', 'I could use ', 'do you have ', 'Do you have ', 'I still need ']

def get_ego_column(r):
    line_ = r['line']
    r['ego'] = 0
    for ele in ego_words_list:
        if ele in line_:
            r['ego'] = 1
            break
    return r


result_df = result_df.apply(get_ego_column, axis=1)

In [113]:
result_df

,gametype,game,player,mode,line,ego
0,Competitive,A10,Lamar,broadcast,[22:33:16] [Craft Scheduler Thread - 14 - Bart...,1
1,Competitive,A10,Alan,whisper,[22:33:23] [Craft Scheduler Thread - 15 - Bart...,1
2,Competitive,A10,Lamar,broadcast,[22:33:41] [Craft Scheduler Thread - 17 - Bart...,0
3,Competitive,A10,Alan,whisper,[22:33:49] [Craft Scheduler Thread - 18 - Bart...,1
4,Competitive,A10,Sokka,broadcast,[22:33:56] [Craft Scheduler Thread - 18 - Bart...,0
...,...,...,...,...,...,...
6606,Cooperative,C8,Sokka,broadcast,[14:41:01] [Craft Scheduler Thread - 166 - Bar...,1
6607,Cooperative,C8,Lamar,broadcast,[14:41:07] [Craft Scheduler Thread - 167 - Bar...,1
6608,Cooperative,C8,Alan,broadcast,[14:41:15] [Craft Scheduler Thread - 168 - Bar...,1
6609,Cooperative,C8,Alan,broadcast,[14:49:38] [Craft Scheduler Thread - 175 - Bar...,1


In [114]:
result_df.groupby(by=['game','player', 'mode']).agg({'line':len, 'ego':np.sum})

C:\Users\Vijay Bhaskar\AppData\Local\Temp\ipykernel_18292\2181509277.py:1: FutureWarning: The provided callable <function sum at 0x0000015A7E926A20> is currently using SeriesGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  result_df.groupby(by=['game','player', 'mode']).agg({'line':len, 'ego':np.sum})


line  ego
game player  mode                
A10  Alan    broadcast     1    0
             whisper      15   11
     Kalyani broadcast    17    6
             whisper      19    3
     Lamar   broadcast    23    5
...                      ...  ...
C8   Lamar   whisper      39   10
     Sarah   broadcast     1    1
             whisper      17   11
     Sokka   broadcast    18    8
             whisper      16    6

[228 rows x 2 columns]

In [115]:
whisper_ego_df = result_df[result_df['mode']==WHISPER]
ego_whisper_count_df = whisper_ego_df.groupby(by=['game','player']).agg({'line':len, 'ego':np.sum})
ego_whisper_count_df.to_excel('ego-whisper-agent-chats-df.xlsx')

broadcast_ego_df = result_df[result_df['mode']==BROADCAST]
ego_broadcast_count_df = broadcast_ego_df.groupby(by=['game','player']).agg({'line':len, 'ego':np.sum})
ego_broadcast_count_df.to_excel('ego-broadcast-agent-chats-df.xlsx')

# ego_count_df = result_df.groupby(by=['game','player', 'mode']).agg({'line':len, 'ego':np.sum})
# ego_count_df.to_excel('ego-count-agent-chats-df.xlsx')

C:\Users\Vijay Bhaskar\AppData\Local\Temp\ipykernel_18292\1811475817.py:2: FutureWarning: The provided callable <function sum at 0x0000015A7E926A20> is currently using SeriesGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  ego_whisper_count_df = whisper_ego_df.groupby(by=['game','player']).agg({'line':len, 'ego':np.sum})
C:\Users\Vijay Bhaskar\AppData\Local\Temp\ipykernel_18292\1811475817.py:6: FutureWarning: The provided callable <function sum at 0x0000015A7E926A20> is currently using SeriesGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  ego_broadcast_count_df = broadcast_ego_df.groupby(by=['game','player']).agg({'line':len, 'ego':np.sum})
